In [23]:
import yfinance as yf
import pandas as pd

# List of 10 tech stocks
tech_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'NFLX', 'ADBE', 'INTC']

# Download data for these stocks
intraday_data = yf.download(tickers=tech_stocks, period='5d', interval='1m')

# Save to CSV for further analysis (optional)
intraday_data.to_csv(f'tech_stocks_intraday_data.csv')


[*********************100%***********************]  10 of 10 completed


In [29]:
adj_close_data = intraday_data['Adj Close']
# Check if there are still missing values
#print(adj_close_data.isnull().sum())
# Display the first few rows
adj_close_data.fillna(method='ffill', inplace=True)
adj_close_data

# Calculate the percentage change for each stock
pct_change_data = adj_close_data.pct_change() * 100  # Convert to percentage

# Display the first few rows of percentage change data
pct_change_data.dropna(inplace=True)

pct_change_data

C:\Users\User\AppData\Local\Temp\ipykernel_28468\3940698456.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adj_close_data.fillna(method='ffill', inplace=True)


Ticker,AAPL,ADBE,AMZN,GOOGL,INTC,META,MSFT,NFLX,NVDA,TSLA
Datetime,,,,,,,,,,
2024-09-23 13:31:00+00:00,-0.068330,0.113992,-0.177188,-0.032072,0.655712,0.129216,-0.179502,0.151439,0.364314,0.783760
2024-09-23 13:32:00+00:00,-0.240421,-0.211601,-0.261035,-0.556799,-0.796207,-0.280924,-0.608642,-0.111986,-0.504829,-0.006228
2024-09-23 13:33:00+00:00,-0.006633,0.244377,0.091603,-0.213006,-0.693826,0.066909,0.104383,0.111885,0.012897,0.437499
2024-09-23 13:34:00+00:00,0.249863,-0.132793,0.026149,0.054889,-0.584172,-0.103819,-0.040547,0.141450,-0.193354,0.279407
2024-09-23 13:35:00+00:00,-0.167632,0.026587,-0.036601,-0.030415,0.061660,-0.074857,-0.148361,-0.059827,-0.112072,-0.022375
...,...,...,...,...,...,...,...,...,...,...
2024-09-27 19:55:00+00:00,0.087822,0.067956,0.021273,0.003053,-0.041728,0.034368,0.065477,-0.036679,0.000000,-0.111333
2024-09-27 19:56:00+00:00,-0.024127,-0.032013,-0.010630,-0.070144,0.062615,-0.024657,-0.045572,-0.099492,-0.007494,0.084561
2024-09-27 19:57:00+00:00,0.046076,0.029120,-0.021279,0.015257,0.062584,-0.029084,0.046763,-0.051558,-0.037792,0.001910


In [30]:
# Use quantiles to classify the data
def classify_quantiles(pct_change, q1, q2, q3, q4):
    if pct_change > q4:
        return 'A+'
    elif pct_change > q3:
        return 'B+'
    elif pct_change > q2:
        return 'C+'
    elif pct_change > q1:
        return 'C-'
    else:
        return 'B-'

# Apply quantile-based classification
classified_data_quantiles = pct_change_data.copy()

for ticker in pct_change_data.columns:
    q1 = pct_change_data[ticker].quantile(0.2)
    q2 = pct_change_data[ticker].quantile(0.4)
    q3 = pct_change_data[ticker].quantile(0.6)
    q4 = pct_change_data[ticker].quantile(0.8)
    
    classified_data_quantiles[ticker] = pct_change_data[ticker].apply(classify_quantiles, args=(q1, q2, q3, q4))

# Display classified data
print(classified_data_quantiles.head())

Ticker                    AAPL ADBE AMZN GOOGL INTC META MSFT NFLX NVDA TSLA
Datetime                                                                    
2024-09-23 13:31:00+00:00   B-   A+   B-    B-   A+   A+   B-   A+   A+   A+
2024-09-23 13:32:00+00:00   B-   B-   B-    B-   B-   B-   B-   B-   B-   C+
2024-09-23 13:33:00+00:00   C+   A+   A+    B-   B-   A+   A+   A+   C+   A+
2024-09-23 13:34:00+00:00   A+   B-   B+    A+   B-   B-   B-   A+   B-   A+
2024-09-23 13:35:00+00:00   B-   B+   B-    B-   B+   B-   B-   B-   B-   C-


In [36]:
def create_variable_sequences(data):
    sequences = []  # List to store the final sequences
    current_seq = []  # List to hold the current sequence of continuous trends
    
    # Loop over the entire dataset (assuming each row is a categorized stock movement)
    for i in range(len(data)):
        # If this is the first data point or the current category is the same as the previous one
        if i == 0 or data['Category'].iloc[i] == data['Category'].iloc[i-1]:
            current_seq.append(data['Category'].iloc[i])  # Append the category to the current sequence
        else:
            # If the current category differs from the previous one, complete the current sequence
            sequences.append(' '.join(current_seq))  # Join the current sequence into a string and store it
            current_seq = [data['Category'].iloc[i]]  # Start a new sequence with the current category
    
    # Return the list of sequences
    return sequences

variable_sequences = create_variable_sequences(classified_data_quantiles)
classified_data_quantiles

Ticker,AAPL,ADBE,AMZN,GOOGL,INTC,META,MSFT,NFLX,NVDA,TSLA,Category
Datetime,,,,,,,,,,,
2024-09-23 13:31:00+00:00,B-,A+,B-,B-,A+,A+,B-,A+,A+,A+,B-
2024-09-23 13:32:00+00:00,B-,B-,B-,B-,B-,B-,B-,B-,B-,C+,B-
2024-09-23 13:33:00+00:00,C+,A+,A+,B-,B-,A+,A+,A+,C+,A+,C+
2024-09-23 13:34:00+00:00,A+,B-,B+,A+,B-,B-,B-,A+,B-,A+,A+
2024-09-23 13:35:00+00:00,B-,B+,B-,B-,B+,B-,B-,B-,B-,C-,B-
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-27 19:55:00+00:00,A+,A+,B+,C+,C-,B+,A+,C-,C+,B-,A+
2024-09-27 19:56:00+00:00,C-,C-,C-,B-,B+,C-,B-,B-,C+,A+,C-
2024-09-27 19:57:00+00:00,A+,B+,C-,B+,B+,C-,A+,B-,C-,C+,A+
